In [1]:
import os
import pandas as pd

# Path to the folder with the CSV files
csv_folder = '../tubitakaiagentprojeleriiinverisetleri/'

import pandas as pd

# Load the datasets
try:
    purchase_df = pd.read_csv(csv_folder+'Purchase.csv')
    before_purchase_df = pd.read_csv(csv_folder+'BeforePurchaseDetailsScreen.csv')
    dog_added_df = pd.read_csv(csv_folder+'DogAdded.csv')
    cat_added_df = pd.read_csv(csv_folder+'CatAdded.csv')
    checkout_opened_df = pd.read_csv(csv_folder+'CheckoutPageOpened.csv')
    address_added_df = pd.read_csv(csv_folder+'AddressAdded.csv')
    sign_up_df = pd.read_csv(csv_folder+'SignUpCompleted.csv')
    creditcard_added_df = pd.read_csv(csv_folder+'CreditcardAdded.csv')

    # Inspect each dataframe
    print("Purchase.csv info and head:")
    purchase_df.info()
    purchase_df.head()

    print("\nBeforePurchaseDetailsScreen.csv info and head:")
    before_purchase_df.info()
    before_purchase_df.head()

    print("\nDogAdded.csv info and head:")
    dog_added_df.info()
    dog_added_df.head()

    print("\nCatAdded.csv info and head:")
    cat_added_df.info()
    cat_added_df.head()

    print("\nCheckoutPageOpened.csv info and head:")
    checkout_opened_df.info()
    checkout_opened_df.head()
    
    print("\nAddressAdded.csv info and head:")
    address_added_df.info()
    address_added_df.head()
    
    print("\nSignUpCompleted.csv info and head:")
    sign_up_df.info()
    sign_up_df.head()
    
    print("\nCreditcardAdded.csv info and head:")
    creditcard_added_df.info()
    creditcard_added_df.head()

except FileNotFoundError as e:
    print(f"Error loading files: {e}. Please ensure all files are uploaded.")
except Exception as e:
    print(f"An error occurred: {e}")

Purchase.csv info and head:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28770 entries, 0 to 28769
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   serviceid         28770 non-null  object
 1   ownerid           28768 non-null  object
 2   ordercreatedtime  28770 non-null  object
 3   servicetype       28770 non-null  object
dtypes: object(4)
memory usage: 899.2+ KB

BeforePurchaseDetailsScreen.csv info and head:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36911 entries, 0 to 36910
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           15209 non-null  object
 1   user_id        15209 non-null  object
 2   event_time     15209 non-null  object
 3   serviceType    15209 non-null  object
 4   uuid.1         21702 non-null  object
 5   user_id.1      21702 non-null  object
 6   event_time.1   21702 non-null  object
 7 

In [2]:
# --- Data Cleaning and Preparation ---

#  Clean BeforePurchaseDetailsScreen.csv
# The file has a strange structure with duplicated columns. We need to merge them.
before_purchase_part1 = before_purchase_df[['uuid', 'user_id', 'event_time', 'serviceType']].dropna()
before_purchase_part2 = before_purchase_df[['uuid.1', 'user_id.1', 'event_time.1', 'serviceType.1']].dropna()
before_purchase_part2.columns = ['uuid', 'user_id', 'event_time', 'serviceType']
before_purchase_df_cleaned = pd.concat([before_purchase_part1, before_purchase_part2], ignore_index=True)
before_purchase_df_cleaned.rename(columns={'user_id': 'ownerid'}, inplace=True)

# Corrigir datas para formato misto
for col in ['event_time']:
    before_purchase_df_cleaned[col] = pd.to_datetime(before_purchase_df_cleaned[col], format='mixed', errors='coerce')

#  Clean CheckoutPageOpened.csv
checkout_opened_df.rename(columns={'user_id': 'ownerid'}, inplace=True)

if 'event_time' in checkout_opened_df.columns:
    checkout_opened_df['event_time'] = pd.to_datetime(checkout_opened_df['event_time'], format='mixed', errors='coerce')

#  Clean Purchase.csv
purchase_df.dropna(subset=['ownerid'], inplace=True)
purchase_df.rename(columns={'servicetype': 'serviceType'}, inplace=True)

if 'event_time' in purchase_df.columns:
    purchase_df['event_time'] = pd.to_datetime(purchase_df['event_time'], format='mixed', errors='coerce')

#  Clean and merge DogAdded.csv and CatAdded.csv
dog_added_df['pet_type'] = 'dog'
dog_added_df.rename(columns={'dogid': 'petid'}, inplace=True)

if 'event_time' in dog_added_df.columns:
    dog_added_df['event_time'] = pd.to_datetime(dog_added_df['event_time'], format='mixed', errors='coerce')

cat_added_df['pet_type'] = 'cat'
cat_added_df.rename(columns={'catid': 'petid'}, inplace=True)

if 'event_time' in cat_added_df.columns:
    cat_added_df['event_time'] = pd.to_datetime(cat_added_df['event_time'], format='mixed', errors='coerce')

pets_df = pd.concat([dog_added_df, cat_added_df], ignore_index=True)

# Owners
owners_df = sign_up_df.dropna(subset=['id']).copy()
owners_df.rename(columns={'id': 'ownerid'}, inplace=True)
owners_df['signuptime'] = pd.to_datetime(owners_df['signuptime'])


#  Clean AddressAdded.csv
addresses_df = address_added_df.dropna(subset=['ownerid', 'addressid'])

if 'event_time' in addresses_df.columns:
    addresses_df['event_time'] = pd.to_datetime(addresses_df['event_time'], format='mixed', errors='coerce')

#  Clean SignUpCompleted.csv
sign_up_df_cleaned = sign_up_df.dropna(subset=['id'])
sign_up_df_cleaned.rename(columns={'id': 'ownerid'}, inplace=True)

if 'event_time' in sign_up_df_cleaned.columns:
    sign_up_df_cleaned['event_time'] = pd.to_datetime(sign_up_df_cleaned['event_time'], format='mixed', errors='coerce')

#  Clean CreditcardAdded.csv
creditcard_added_df_cleaned = creditcard_added_df.dropna(subset=['ownerid', 'creditcardid'])

if 'event_time' in creditcard_added_df_cleaned.columns:
    creditcard_added_df_cleaned['event_time'] = pd.to_datetime(creditcard_added_df_cleaned['event_time'], format='mixed', errors='coerce')

# --- Merging DataFrames ---

# Merge checkout and purchase data to create a conversion funnel
# We are considering a conversion if a user who opened the checkout page for a service type made a purchase for the same service type.
# Aggregate checkout and purchase data
checkout_agg = checkout_opened_df.groupby(['ownerid', 'serviceType']).size().reset_index(name='checkout_count')
purchase_agg = purchase_df.groupby(['ownerid', 'serviceType']).size().reset_index(name='purchase_count')

# Merge for conversion analysis
conversion_df = pd.merge(checkout_agg, purchase_agg, on=['ownerid', 'serviceType'], how='left')
conversion_df['purchase_count'].fillna(0, inplace=True)
conversion_df['converted'] = conversion_df['purchase_count'] > 0


C:\Users\caioa\AppData\Local\Temp\ipykernel_11536\3220573907.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  conversion_df['purchase_count'].fillna(0, inplace=True)


In [3]:
# --- Analysis and Visualization ---

# 1. Overall Conversion Rate
total_checkouts = len(checkout_agg)
total_conversions = len(conversion_df[conversion_df['converted']])
overall_conversion_rate = (total_conversions / total_checkouts) * 100

print(f"Overall Conversion Rate: {overall_conversion_rate:.2f}%")


Overall Conversion Rate: 43.92%


In [4]:
print("\n" + "="*80)
print("ANALYSIS 1: CUSTOMER CONVERSION FUNNEL")
print("="*80)
print("This funnel shows where users drop off between signing up and making a purchase.")

# Get unique owner IDs at each step
signed_up_users = set(owners_df['ownerid'])
added_pet_users = set(pets_df['ownerid'])
added_address_users = set(addresses_df['ownerid'])
added_card_users = set(creditcard_added_df['ownerid'])
opened_checkout_users = set(checkout_opened_df['ownerid'])
purchased_users = set(purchase_df['ownerid'])

# Build the funnel data
funnel_data = {
'Step': ["1. Signed Up", "2. Added a Pet", "3. Added Address", "4. Added Credit Card", "5. Opened Checkout", "6. Made Purchase"],
'User Count': [
    len(signed_up_users),
    len(added_pet_users),
    len(added_address_users),
    len(added_card_users),
    len(opened_checkout_users),
    len(purchased_users)
]
}
funnel_df = pd.DataFrame(funnel_data)
funnel_df['Overall Conversion from Signup'] = (funnel_df['User Count'] / funnel_df['User Count'].iloc[0] * 100).map('{:.2f}%'.format)

print(funnel_df.to_string(index=False))





ANALYSIS 1: CUSTOMER CONVERSION FUNNEL
This funnel shows where users drop off between signing up and making a purchase.
                Step  User Count Overall Conversion from Signup
        1. Signed Up       10986                        100.00%
      2. Added a Pet        1888                         17.19%
    3. Added Address        1771                         16.12%
4. Added Credit Card         643                          5.85%
  5. Opened Checkout        3213                         29.25%
    6. Made Purchase        1645                         14.97%


In [5]:
print("\n" + "="*80)
print("ANALYSIS 2: TOP PERFORMING SEGMENTS")
print("="*80)
print("This analysis identifies the most valuable services, pet types, and locations.")

# Merge purchases with pet and address data to get richer context
purchases_with_details = pd.merge(purchase_df, pets_df[['ownerid', 'pet_type']].drop_duplicates(), on='ownerid', how='left')
purchases_with_details = pd.merge(purchases_with_details, addresses_df[['ownerid', 'province']].drop_duplicates(), on='ownerid', how='left')

# Top 5 most purchased services
top_services = purchases_with_details['serviceType'].value_counts().nlargest(5)
print("\n--- Top 5 Most Purchased Services ---")
print(top_services.to_string())
print("\nINSIGHT: 'Customize', 'AdHoc' and 'Planned' are the most popular services.")
print("ACTION: Focus marketing campaigns and promotions on these high-demand services.")

# Sales by Pet Type
sales_by_pet_type = purchases_with_details.groupby('pet_type')['serviceid'].count().sort_values(ascending=False)
print("\n--- Total Purchases by Pet Type ---")
print(sales_by_pet_type.to_string())
print("\nINSIGHT: Dog owners make significantly more purchases than cat owners.")
print("ACTION: Create marketing campaigns specifically targeting cat owners to boost their engagement.")
print("         For dog owners, consider loyalty programs to retain these high-value customers.")

# Top 5 provinces by sales
sales_by_province = purchases_with_details['province'].value_counts().nlargest(5)
print("\n--- Top 5 Provinces by Number of Purchases ---")
print(sales_by_province.to_string())
print("\nINSIGHT: Sales are heavily concentrated in İstanbul.")
print("ACTION: Double down on marketing efforts in İstanbul. For other cities like İzmir and Ankara,")
print("         consider targeted local campaigns to increase market share.")



ANALYSIS 2: TOP PERFORMING SEGMENTS
This analysis identifies the most valuable services, pet types, and locations.

--- Top 5 Most Purchased Services ---
serviceType
Customize      15843
AdHoc           7346
Planned         4812
WalkAndCare      999
Boarding         219

INSIGHT: 'Customize', 'AdHoc' and 'Planned' are the most popular services.
ACTION: Focus marketing campaigns and promotions on these high-demand services.

--- Total Purchases by Pet Type ---
pet_type
dog    3302
cat     156

INSIGHT: Dog owners make significantly more purchases than cat owners.
ACTION: Create marketing campaigns specifically targeting cat owners to boost their engagement.
         For dog owners, consider loyalty programs to retain these high-value customers.

--- Top 5 Provinces by Number of Purchases ---
province
İstanbul    4785
İzmir        946
Ankara       146
Mersin        16
Muğla          6

INSIGHT: Sales are heavily concentrated in İstanbul.
ACTION: Double down on marketing efforts in İstan

In [6]:
print("\n" + "="*80)
print("ANALYSIS 3: TIME FROM SIGNUP TO FIRST PURCHASE")
print("="*80)
print("This analysis shows how long it takes for a new user to become a paying customer.")

# Merge owner signup time with their purchases
time_analysis_df = pd.merge(owners_df, purchase_df, on='ownerid')

# Garantir que ambas as colunas são datetime
time_analysis_df['ordercreatedtime'] = pd.to_datetime(time_analysis_df['ordercreatedtime'], errors='coerce')
time_analysis_df['signuptime'] = pd.to_datetime(time_analysis_df['signuptime'], errors='coerce')

# Remover linhas com NaN em 'ordercreatedtime' ou 'ownerid'
time_analysis_df = time_analysis_df.dropna(subset=['ordercreatedtime', 'ownerid'])

# Find the first purchase time for each user
first_purchase_df = time_analysis_df.loc[time_analysis_df.groupby('ownerid')['ordercreatedtime'].idxmin()]

# Calculate time to purchase
first_purchase_df['time_to_purchase'] = first_purchase_df['ordercreatedtime'] - first_purchase_df['signuptime']

# Remove negative deltas which can result from data errors
first_purchase_df = first_purchase_df[first_purchase_df['time_to_purchase'] >= pd.Timedelta(0)]

avg_time_to_purchase = first_purchase_df['time_to_purchase'].mean()
median_time_to_purchase = first_purchase_df['time_to_purchase'].median()

print(f"\nAverage time from signup to first purchase: {avg_time_to_purchase.days} days")
print(f"Median time from signup to first purchase:  {median_time_to_purchase.days} days")
print("\nINSIGHT: The median time to purchase is a key metric. Half of all new customers convert within this timeframe.")
print("ACTION: Consider setting up an automated marketing campaign (e.g., a special offer email)")
print(f"         for users who haven't made a purchase after {median_time_to_purchase.days + 1} days to re-engage them.")




ANALYSIS 3: TIME FROM SIGNUP TO FIRST PURCHASE
This analysis shows how long it takes for a new user to become a paying customer.

Average time from signup to first purchase: 2 days
Median time from signup to first purchase:  0 days

INSIGHT: The median time to purchase is a key metric. Half of all new customers convert within this timeframe.
ACTION: Consider setting up an automated marketing campaign (e.g., a special offer email)
         for users who haven't made a purchase after 1 days to re-engage them.


In [7]:
min_owners_per_breed = 10  # Minimum number of owners per breed to consider it significant

purchasing_users = set(purchase_df['ownerid'].dropna().unique())

print("\n" + "="*80)
print(f"ANALYSIS: TOP CONVERTING DOG BREEDS (min. {min_owners_per_breed} owners)")
print("="*80)

# Clean and prepare dog breed data
dog_breeds_df = dog_added_df[['ownerid', 'breed']].dropna().copy()
dog_breeds_df['breed'] = dog_breeds_df['breed'].str.strip().str.lower()
dog_owner_breeds = dog_breeds_df.drop_duplicates()

# Flag owners who have converted (made a purchase)
dog_owner_breeds['converted'] = dog_owner_breeds['ownerid'].isin(purchasing_users)

# Calculate conversion stats for each breed
dog_breed_stats = dog_owner_breeds.groupby('breed')['converted'].agg(
    total_owners='size',
    converted_owners='sum'
).reset_index()

# Filter for breeds that meet the minimum owner threshold
significant_dog_breeds = dog_breed_stats[dog_breed_stats['total_owners'] >= min_owners_per_breed].copy()

# Calculate conversion rate and sort
significant_dog_breeds['conversion_rate'] = (significant_dog_breeds['converted_owners'] / significant_dog_breeds['total_owners']) * 100
top_dog_breeds = significant_dog_breeds.sort_values(by='conversion_rate', ascending=False)
top_dog_breeds['conversion_rate'] = top_dog_breeds['conversion_rate'].map('{:.2f}%'.format)

# Display the results
if top_dog_breeds.empty:
    print(f"\nNo dog breeds met the minimum owner threshold of {min_owners_per_breed}.")
else:
    print(top_dog_breeds.to_string(index=False))



ANALYSIS: TOP CONVERTING DOG BREEDS (min. 10 owners)
                         breed  total_owners  converted_owners conversion_rate
                english setter            17                 6          35.29%
 cavalier king charles spaniel            12                 4          33.33%
                     chow chow            19                 6          31.58%
            labrador retriever            64                18          28.12%
                cocker spaniel            15                 4          26.67%
                        beagle            16                 4          25.00%
                french bulldog            57                14          24.56%
             doberman pinscher            21                 5          23.81%
                       mix irk           153                36          23.53%
                siberian husky            15                 3          20.00%
          jack russell terrier            41                 8          19.51

C:\Users\caioa\AppData\Local\Temp\ipykernel_11536\2148076246.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dog_owner_breeds['converted'] = dog_owner_breeds['ownerid'].isin(purchasing_users)


In [8]:
print("\n" + "="*80)
print(f"ANALYSIS: TOP CONVERTING CAT BREEDS (min. {min_owners_per_breed} owners)")
print("="*80)

# Clean and prepare cat breed data
cat_breeds_df = cat_added_df[['ownerid', 'breed']].dropna().copy()
cat_breeds_df['breed'] = cat_breeds_df['breed'].str.strip().str.lower()
cat_owner_breeds = cat_breeds_df.drop_duplicates()

# Flag owners who have converted
cat_owner_breeds['converted'] = cat_owner_breeds['ownerid'].isin(purchasing_users)

# Calculate conversion stats for each breed
cat_breed_stats = cat_owner_breeds.groupby('breed')['converted'].agg(
    total_owners='size',
    converted_owners='sum'
).reset_index()

# Filter for breeds that meet the minimum owner threshold
significant_cat_breeds = cat_breed_stats[cat_breed_stats['total_owners'] >= min_owners_per_breed].copy()

# Calculate conversion rate and sort
significant_cat_breeds['conversion_rate'] = (significant_cat_breeds['converted_owners'] / significant_cat_breeds['total_owners']) * 100
top_cat_breeds = significant_cat_breeds.sort_values(by='conversion_rate', ascending=False)
top_cat_breeds['conversion_rate'] = top_cat_breeds['conversion_rate'].map('{:.2f}%'.format)

# Display the results
if top_cat_breeds.empty:
    print(f"\nNo cat breeds met the minimum owner threshold of {min_owners_per_breed}.")
else:
    print(top_cat_breeds.to_string(index=False))

# --- 4. Final Insights ---
print("\n\n" + "="*80)
print("INSIGHT: By separating dogs and cats, you can identify the highest-value breeds within each category.")
print("ACTION: Tailor your marketing strategies. For example, run a Facebook ad campaign targeting 'Golden Retriever' owners")
print("         with ads for 'Boarding' services, and another campaign for 'British Shorthair' owners focusing on 'Grooming'.")
print("="*80)


ANALYSIS: TOP CONVERTING CAT BREEDS (min. 10 owners)
                   breed  total_owners  converted_owners conversion_rate
            sokak dedisi            12                 4          33.33%
           scottish fold            34                 6          17.65%
        uzun tüylü tekir            18                 3          16.67%
üç renkli tekir (calico)            26                 4          15.38%
            sokak kedisi            13                 2          15.38%
                   tekir           103                13          12.62%
                  sarman            32                 4          12.50%
                  smokin            18                 2          11.11%
       british shorthair            76                 6           7.89%
                  bombay            17                 1           5.88%
        british longhair            21                 1           4.76%


INSIGHT: By separating dogs and cats, you can identify the highest-v

C:\Users\caioa\AppData\Local\Temp\ipykernel_11536\858673676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_owner_breeds['converted'] = cat_owner_breeds['ownerid'].isin(purchasing_users)


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns


In [10]:
print("="*80)
print("Implementing Technique 1: Logistic Regression to Predict Customer Conversion")
print("="*80)

users_df = sign_up_df.rename(columns={'id': 'ownerid'})
users_df['signuptime'] = pd.to_datetime(users_df['signuptime'])

# --- 1b. Feature Engineering ---
print("\nStep 2: Engineering features for the model...")

# Target Variable: Did the user make a purchase?
purchasing_users = set(purchase_df['ownerid'].unique())
users_df['has_purchased'] = users_df['ownerid'].isin(purchasing_users).astype(int)

# Feature 1: Did the user add a pet?
pets_df = pd.concat([dog_added_df[['ownerid']], cat_added_df[['ownerid']]])
added_pet_users = set(pets_df['ownerid'].unique())
users_df['has_pet'] = users_df['ownerid'].isin(added_pet_users).astype(int)

# Feature 2: Did the user add an address?
added_address_users = set(address_added_df['ownerid'].unique())
users_df['has_address'] = users_df['ownerid'].isin(added_address_users).astype(int)

# Feature 3: Did the user add a credit card?
added_card_users = set(creditcard_added_df['ownerid'].unique())
users_df['has_credit_card'] = users_df['ownerid'].isin(added_card_users).astype(int)

# Feature 4: User tenure (how long they've been signed up)
# We use the latest signup time as 'today' to make the calculation stable
latest_signup = users_df['signuptime'].max()
users_df['tenure_days'] = (latest_signup - users_df['signuptime']).dt.days

# Combine all features into a single model-ready dataframe
features = ['has_pet', 'has_address', 'has_credit_card', 'tenure_days']
target = 'has_purchased'

model_df = users_df[features + [target]].copy()
model_df.dropna(inplace=True)

X = model_df[features]
y = model_df[target]

print(f"Created a dataset with {X.shape[0]} users and {X.shape[1]} features.")

# --- 1c. Train the Logistic Regression Model ---
print("\nStep 3: Training the Logistic Regression model...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)
print("Model training complete.")

# --- 1d. Evaluate the Model ---
print("\nStep 4: Evaluating the model's performance...")
y_pred = log_reg.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.2%}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("(Rows: Actual, Columns: Predicted)")

# --- 1e. Interpret the Results ---
print("\nStep 5: Interpreting model coefficients...")
coefficients = pd.DataFrame(log_reg.coef_[0], X.columns, columns=['Coefficient'])
coefficients.sort_values('Coefficient', ascending=False, inplace=True)

print("Feature importance (higher coefficient means stronger predictor of purchase):")
print(coefficients)


Implementing Technique 1: Logistic Regression to Predict Customer Conversion

Step 2: Engineering features for the model...
Created a dataset with 10986 users and 4 features.

Step 3: Training the Logistic Regression model...
Model training complete.

Step 4: Evaluating the model's performance...

Model Accuracy: 99.54%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3239
           1       0.96      0.77      0.85        57

    accuracy                           1.00      3296
   macro avg       0.98      0.89      0.93      3296
weighted avg       1.00      1.00      1.00      3296


Confusion Matrix:
[[3237    2]
 [  13   44]]
(Rows: Actual, Columns: Predicted)

Step 5: Interpreting model coefficients...
Feature importance (higher coefficient means stronger predictor of purchase):
                 Coefficient
has_pet             0.945980
has_address         0.892878
has_credit_card     0.696328
tenure_d

In [11]:
print("\n\n" + "="*80)
print("Implementing Technique 2: K-Means Clustering to Find Customer Segments")
print("="*80)

# --- 2a. Load and Prepare Data for Clustering ---
print("\nStep 1: Preparing data for clustering (using paying customers only)...")
# We only cluster users who have made at least one purchase
customers_df = purchase_df.copy()
customers_df['ordercreatedtime'] = pd.to_datetime(customers_df['ordercreatedtime'], errors='coerce', utc=True)

# --- 2b. Feature Engineering for RFM-like analysis ---
print("\nStep 2: Engineering features for customer segmentation...")
latest_purchase_date = customers_df['ordercreatedtime'].max()

# Calculate Recency, Frequency, and Tenure (as a proxy for Monetary)
segmentation_df = customers_df.groupby('ownerid').agg(
    recency=('ordercreatedtime', lambda date: (latest_purchase_date - date.max()).days),
    frequency=('serviceid', 'count'),
    unique_services=('serviceType', 'nunique')
).reset_index()

print(f"Created a dataset of {segmentation_df.shape[0]} customers.")

# --- 2c. Find the Optimal Number of Clusters (Elbow Method) ---
print("\nStep 3: Using the Elbow Method to find the optimal 'K'...")
features_for_clustering = ['recency', 'frequency', 'unique_services']
X_cluster = segmentation_df[features_for_clustering]

# Remover linhas com NaN nas features antes de rodar o KMeans
X_cluster = X_cluster.dropna()

# Scale the features
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# Calculate inertia for a range of K values
inertia = []
K_range = range(1, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X_cluster_scaled)
    inertia.append(kmeans.inertia_)

optimal_k = 4

# --- 2d. Run K-Means and Analyze Segments ---
print(f"\nStep 4: Running K-Means with K={optimal_k} and analyzing segments...")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto')
cluster_labels = kmeans.fit_predict(X_cluster_scaled)

# Adicionar os rótulos de cluster ao DataFrame de segmentação (apenas para os clientes sem NaN)
segmentation_df_clean = segmentation_df.loc[X_cluster.index].copy()
segmentation_df_clean['cluster'] = cluster_labels

# Analyze the characteristics of each cluster
cluster_analysis = segmentation_df_clean.groupby('cluster')[features_for_clustering].mean().round(2)
cluster_analysis['customer_count'] = segmentation_df_clean['cluster'].value_counts()
print("\n--- Customer Segment Profiles ---")
print(cluster_analysis)





Implementing Technique 2: K-Means Clustering to Find Customer Segments

Step 1: Preparing data for clustering (using paying customers only)...

Step 2: Engineering features for customer segmentation...
Created a dataset of 1645 customers.

Step 3: Using the Elbow Method to find the optimal 'K'...
Created a dataset of 1645 customers.

Step 3: Using the Elbow Method to find the optimal 'K'...

Step 4: Running K-Means with K=4 and analyzing segments...

--- Customer Segment Profiles ---
         recency  frequency  unique_services  customer_count
cluster                                                     
0          17.82      18.39             2.28             535
1          18.12       9.15             1.00             481
2          67.92       6.55             1.19             505
3          18.84     120.31             2.08              93

Step 4: Running K-Means with K=4 and analyzing segments...

--- Customer Segment Profiles ---
         recency  frequency  unique_services  cu

In [19]:
from mlxtend.frequent_patterns import apriori, association_rules

print("="*80)
print("Implementing Technique 3: Market Basket Analysis to Find Service Bundles")
print("="*80)

basket_df = purchase_df[['ownerid', 'serviceType']].dropna().drop_duplicates()


basket_encoded = basket_df.pivot_table(index='ownerid', columns='serviceType', aggfunc='size', fill_value=0)

# We only care if the service was purchased, not how many times, so we'll convert counts to 1s.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_encoded = basket_encoded.map(encode_units)
print("Data successfully one-hot encoded.")
print(f"Created a transaction matrix with {basket_encoded.shape[0]} owners and {basket_encoded.shape[1]} unique services.")


# --- 3c. Run the Apriori Algorithm ---
# This step finds "frequent itemsets" - sets of services that are purchased together more
# often than a specified threshold (min_support).
print("\nStep 3: Running the Apriori algorithm to find frequent itemsets...")

# Set the minimum support threshold. 1% (0.01) is a common starting point.
# This means we are looking for itemsets that appear in at least 1% of all transactions.
# You can adjust this value: lower it to find more, potentially less significant, itemsets,
# or raise it to focus only on the most common combinations.
min_support_threshold = 0.01

frequent_itemsets = apriori(basket_encoded, min_support=min_support_threshold, use_colnames=True)
frequent_itemsets.sort_values('support', ascending=False, inplace=True)

print(f"Found {len(frequent_itemsets)} frequent itemsets with support >= {min_support_threshold:.0%}")
print("\nTop 10 most frequent itemsets (services bought together):")
print(frequent_itemsets.head(10))


# --- 3d. Generate and Interpret Association Rules ---
# Now we take the frequent itemsets and generate rules like "If a user buys X, they will also buy Y."
print("\n\nStep 4: Generating and interpreting association rules...")

# We generate rules based on 'lift', a metric that measures how much more likely
# a user is to buy the `consequents` (e.g., 'Grooming') if they have already bought
# the `antecedents` (e.g., 'Boarding'). A lift > 1 is generally considered interesting.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Sort the rules by confidence and lift to find the strongest relationships.
rules.sort_values(['confidence', 'lift'], ascending=[False, False], inplace=True)

print(f"\nGenerated {len(rules)} association rules with lift >= 1.")
print("\n--- Top Association Rules ---")
print("(Read as: 'If a user buys the services in `antecedents`, they are likely to also buy the ones in `consequents`')")

# Display the most relevant columns for easier interpretation
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10).to_string())

print("\n\n--- How to Interpret the Results ---")
print("support: The percentage of all customers who purchased this combination of services.")
print("confidence: If a customer buys the `antecedent`, what's the probability they also buy the `consequent`?")
print("  (e.g., A confidence of 0.6 means 60% of people who bought the first item also bought the second).")
print("lift: How much more likely a customer is to buy the `consequent` given they bought the `antecedent`.")
print("  (Lift > 1 suggests a strong association).")

print("Create Bundles: Look for rules with high confidence and lift.")

Implementing Technique 3: Market Basket Analysis to Find Service Bundles
Data successfully one-hot encoded.
Created a transaction matrix with 1645 owners and 12 unique services.

Step 3: Running the Apriori algorithm to find frequent itemsets...
Found 22 frequent itemsets with support >= 1%

Top 10 most frequent itemsets (services bought together):
     support                     itemsets
0   0.540426                      (AdHoc)
5   0.475380                    (Planned)
11  0.266261             (AdHoc, Planned)
3   0.226748                  (Customize)
16  0.113070         (Customize, Planned)
1   0.085106                   (Boarding)
8   0.075380                (WalkAndCare)
10  0.066869           (AdHoc, Customize)
6   0.054711                    (Sitting)
21  0.052888  (AdHoc, Customize, Planned)


Step 4: Generating and interpreting association rules...

Generated 14 association rules with lift >= 1.

--- Top Association Rules ---
(Read as: 'If a user buys the services in `antece

c:\Users\caioa\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
